# Thermal storage model calibration and validation


### Calibrated parameters

### Version 202403 (WIP)

- UA_h: []
- UA_c: []
- Vi_h: []
- Vi_c: []

#### Version 20230714

- UA_h: [0.0069818 , 0.00584034, 0.03041486]
- UA_c: [0.01396848, 0.0001    , 0.02286885]
- Vi_h: [5.94771006, 4.87661781, 2.19737023]
- Vi_c: [5.33410037, 7.56470594, 0.90547187]

#### Original version YYYYMMDD

- UA_h: [0.0069818 , 0.00584034, 0.03041486]
- UA_c: [0.01396848, 0.0001    , 0.02286885]
- Vi_h: [5.94771006, 4.87661781, 2.19737023]
- Vi_c: [5.33410037, 7.56470594, 0.90547187]

Notable changes:

- Re-calibrate model since qts,src is not valid, now calculated from heat exchanger energy balance
- Refactor notebook to use the new `phd_visualizations` package and latest implementation practices.

![](../../docs/models/attachments/solarMED_optimization-Storage model.drawio.svg)

In [1]:
from pathlib import Path
import time
import numpy as np
import pandas as pd
from iapws import IAPWS97 as w_props
import hjson
# from utils import filter_nan, get_Q_from_3wv_model
from loguru import logger

# Visualization packages
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

from solarmed_modeling.utils import data_preprocessing, data_conditioning

# auto reload modules
%load_ext autoreload
%autoreload 2

# Paths definition
output_path: Path = Path("../../docs/models/attachments")
data_path: Path = Path("../../data")

date_str: str = '20230703' #'20230707_20230710'
filenames_process_data = [f'datasets/{date_str}_solarMED.csv', f'datasets/{date_str}_MED.csv']


# Validation. Options:
# 20230505_solarMED
# 20230508_solarMED
# 20230511_solarMED
# 20230628_solarMED
# 20230629_solarMED
# 20230630_solarMED
# 20230703_solarMED
# 20231030_solarMED
# data_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/data') / '20230707_20230710_solarMED.csv'

sample_rate = '5s'
sample_rate_numeric = int(sample_rate[:-1])

# Resample figures using plotly_resampler
resample_figures: bool = False

Th_labels: list[str] = ['Tts_h_t', 'Tts_h_m', 'Tts_h_b']
Tc_labels: list[str] = ['Tts_c_t', 'Tts_c_m', 'Tts_c_b']

## Pre-processing

In [2]:
data_paths = [data_path / filename_process_data for filename_process_data in filenames_process_data]

with open( data_path / "variables_config.hjson") as f:
    vars_config = hjson.load(f)
    
# Load data and preprocess data
df = data_preprocessing(data_paths, vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(df, sample_rate_numeric=sample_rate_numeric, vars_config=vars_config)


2024-10-09 08:02:46.326 | INFO     | solarmed_modeling.utils:data_preprocessing:112 - Reading data from 20230703_solarMED.csv


2024-10-09 08:02:46.491 | DEBUG    | solarmed_modeling.utils:process_dataframe:68 - Index([], dtype='object')
2024-10-09 08:02:46.493 | INFO     | solarmed_modeling.utils:process_dataframe:75 - Number of duplicate index values in df: 0
2024-10-09 08:02:46.518 | INFO     | solarmed_modeling.utils:data_preprocessing:132 - Reading data from 20230703_MED.csv
2024-10-09 08:02:46.815 | DEBUG    | solarmed_modeling.utils:process_dataframe:68 - Index([], dtype='object')
2024-10-09 08:02:46.816 | INFO     | solarmed_modeling.utils:process_dataframe:75 - Number of duplicate index values in df: 0
2024-10-09 08:02:46.821 | DEBUG    | solarmed_modeling.utils:data_preprocessing:151 - Common columns in both dataframes: Index(['FT-AQU-100', 'TT-AQU-107a', 'CT-DES-001', 'FT-DES-003', 'FT-DES-005',
       'SW2TC1', 'TE-DES-015', 'TT-DES-030'],
      dtype='object'), dropping them from the auxiliary dataframe.
2024-10-09 08:02:46.871 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:79 -

## Test visualization

In [3]:
plt_config = {
      # General plot attributes
      "title": "Thermal storage",
      "subtitle": "experimental data visualization",
      "height": 1000,
      "width": 1000,
      "margin": {
        "l":20,
        "r":100,
        "t":100,
        "b":20,
        "pad":5
      },
      "vertical_spacing": 0.03,
      "xdomain": [0, 0.85],
      "arrow_xrel_pos": 60, # seconds
    
        # Individual plot attributes
      "plots": {
          "thermal_storage_flows": {
          "title": "<b>Thermal storage</b>",
          "row_height": 0.6,
          "bg_color": "bg_gray", # bg gray
          "ylabels_left": ["m<sup>3</sup>/h"],
          "ylims_left": 'manual',
    
          "traces_left": [
            {
              "var_id": "qts_dis",
              "name": "q<sub>ts,dis</sub>",
              "mode": "lines",
              "color": "plotly_blue",
              "width": 3,
            },
            {
              "var_id": "qts_dis_original",
              "name": "q<sub>ts,dis,measured</sub>",
              "color": "plotly_blue", "width": 1.5, "dash": "dash"
            },
            {
              "var_id": "qts_src", #"qts_src",
              "name": "q<sub>ts,src</sub>",
              "mode": "lines",
              "color": "plotly_red",
              "width": 3,
            },
            {
              "var_id": "qts_src_original", #"qts_src",
              "name": "q<sub>ts,src,measured</sub>",
              "mode": "lines",
              "color": "plotly_red",
              "dash": "dash",
              "width": 1.5,
            },
          ],
    
          # "TODO": Estaría guay añadir trazas a la derecha con power balance
        },
    
        "thermal_storage_power_balance": {
          "row_height": 0.6,
          "title": "Power balance",
          "bg_color": "bg_gray", # bg gray
          "ylabels_left": ["kW<sub>th</sub>"],
          "ylims_left": [0, 260],
    
          "traces_left": [
            # Make this a filled trace between source and discharge, where the color is red when there is more discharge than recharge, and green otherwise
            {
              "var_id": "Pts_src",
              "name": "P<sub>ts,src</sub>",
              "mode": "lines",
              "color": "plotly_yellow",
              "width": 3,
              "fill_between": "Pts_dis", # Traces to plot fill between need to be defined next to each other
            },
            {
              "var_id": "Pts_dis",
              "name": "P<sub>ts,dis</sub>",
              "mode": "lines",
              "color": "wct_purple",
              "width": 3,
              "fill_between": "Pts_src", # Traces to plot fill between need to be defined next to each other
            },
          ]
        },
    
        "thermal_storage_temperatures": {
          "row_height": 1,
          "title": "Hot tank",
          "bg_color": "bg_blue", # bg gray
          # "ylabels_left": ["T<sub>wct</sub> (ºC)"],
          "ylabels_left": ["⁰C"],
          "ylims_left": [60, 100],
          "tight_vertical_spacing": True,
    
          "traces_left": [
            # Hot tank
            {
              "var_id": "Tts_h_t",
              "mode": "lines",
              "color": "plotly_red", # gray
              "width": 3,
            },
            {
              "var_id": "Tts_h_m",
              "mode": "lines",
              "dash": "dash",
              "color": "plotly_red", # gray
              "width": 3,
            },
            {
              "var_id": "Tts_h_b",
              "mode": "lines",
              "color": "plotly_red", # gray
              "dash": "dot",
              "width": 3,
            },
    
            # To MED
            {
              "var_id": "Tts_h_in",
              "mode": "markers",
              "color": "plotly_yellow",
              "width": 3,
              "opacity": 0.5,
              "conditional":{
                  "var_id": "qts_src",
                  "operator": ">",
                  "threshold_value": 1,
              }
            },
    
            # From solar field
            {
              "var_id": "Tts_h_out",
              "mode": "markers",
              "color": "wct_purple",
              "width": 3,
              "opacity": 0.5,
              "conditional":{
                  "var_id": "qts_dis",
                  "operator": ">",
                  "threshold_value": 1,
              }
            },        
          ]
    
          # "TODO": Estaría guay añadir una traza a la derecha con la evolución de 
          # la energía disponible (línea con área), calculada relativa a la Ts_in de la MED
        },
    
        "thermal_storage_temperatures_cold": {
          "row_height": 1,
          "title": "Cold tank",
          "bg_color": "bg_blue", # bg gray
          "ylabels_left": ["⁰C"],
          "ylims_left": [60, 100],
    
    
          "traces_left": [
    
            # Cold tank
            {
              "var_id": "Tts_c_t",
              "mode": "lines",
              "color": "plotly_green", # gray
              "width": 3,
            },
            {
              "var_id": "Tts_c_m",
              "mode": "lines",
              "dash": "dash",
              "color": "plotly_green",
              "width": 3,
            },
            {
              "var_id": "Tts_c_b",
              "mode": "lines",
              "color": "plotly_green",
              "dash": "dot",
              "width": 3,
            },
    
            # From MED
            {
              "var_id": "Tts_c_in",
              "mode": "markers",
              "color": "c_blue",
              "width": 3,
              "opacity": 0.5,
              "conditional":{
                  "var_id": "qts_dis",
                  "operator": ">",
                  "threshold_value": 1,
              }
            },
          ]
    
          # TODO: Estaría guay añadir una traza a la derecha con la evolución de
          # la energía disponible (línea con área), calculada relativa a la Ts_in de la MED
        },
    }
}

In [ ]:

fig = experimental_results_plot(plt_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'thermal_storage_visualization_{df.index[0].strftime("%Y%m%d")}')
)

In [5]:
# save_figure(
#     figure_name=f"thermal_storage_visualization_{df.index[0].strftime('%Y%m%d')}", 
#     figure_path=Path("/home/jmserrano/Downloads"),
#     fig=fig, formats=('png', ), 
#     width=fig.layout.width, height=fig.layout.height, scale=2
# )

In [4]:
from solarmed_modeling.thermal_storage import thermal_storage_two_tanks_model

## Model calibration

In [ ]:
# Calibrate model parameters

from solarmed_modeling.calibration.parameters_fit import objective_function
from optimparallel import minimize_parallel

idx_start = 0
# idx_start = df.index.get_loc(pd.Timestamp('2023-07-09 07:30:00', tz='UTC'))
idx_end = len(df)

# Parameters
Tmin = 60 # Minimum useful temperature, °C
N = len(Th_labels) # Number of volumes
V = 15 # Volume of an individual tank, m³
Vt = 15*2 # Total volume of the storage system (V tank·N tanks)

# Inputs 
Ti_ant_h = np.array( [df.iloc[idx_start][T] for T in Th_labels] )
Ti_ant_c = np.array( [df.iloc[idx_start][T] for T in Tc_labels] )
# Since we are using the first output as starting point, start from the second value
Tt_in = df["Tts_h_in"].values[idx_start+1:idx_end]
Tb_in = df["Tts_c_in"].values[idx_start+1:idx_end]

Tamb = df["Tamb"].values[idx_start+1:idx_end]
qsrc = df["qts_src"].values[idx_start+1:idx_end]
qdis = df["qts_dis"].values[idx_start+1:idx_end]

# Experimental outputs
Ti_ref = np.concatenate((df[Th_labels].values[idx_start+1:], df[Tc_labels].values[idx_start+1:]), axis=1)

# Define optimizer inputs
inputs = [Ti_ant_h, Ti_ant_c, Tt_in, Tb_in, Tamb, qsrc, qdis]  # Input values
outputs = Ti_ref  # Actual output values
params = (sample_rate_numeric, Tmin, Vt)    # Constant model parameters
params_objective_function = {'metric': 'IAE', 'recursive':True, 'n_outputs':2, 
                             'n_parameters': 4} # 'len_outputs':[N, N]

# Set initial parameter values
# initial_parameters = [0.01 for _ in range(N)]
initial_parameters = np.concatenate((np.array([0.0069818 , 0.00584034, 0.03041486]), # UA_h
                                     np.array([0.01396848, 0.0001    , 0.02286885]), # UA_c
                                     np.array([5.94771006, 4.87661781, 2.19737023]), # np.ones(N)*V/N, # Vi_h,
                                     np.array([5.33410037, 7.56470594, 0.90547187]),# np.ones(N)*V/N, # Vi_c
                                    ))
#         UAmin, UAmax    UAmin, UAmax    Vi_min     Vi_max       Vi_min     Vi_max
bounds = ((1e-4, 1),)*N + ((1e-4, 1),)*N + ((0.1*V/N, 2*V/N),)*N + ((0.1*V/N, 2*V/N),)*N

# Perform parameter calibration
optimized_parameters = minimize_parallel(
    objective_function,
    initial_parameters,
    args=(thermal_storage_two_tanks_model, inputs, outputs, params, params_objective_function),
    bounds = bounds,
    # method='Nelder-Mead',
    # method='L-BFGS-B'
).x

op = optimized_parameters

L = int(len(op)/4)
UA_h  = op[:L]
UA_c  = op[L:2*L]
Vi_h  = op[2*L:3*L]
Vi_c  = op[3*L:]

# Print results
logger.info(f"Optimized parameters: {op}")
logger.info(f"UA_h: {UA_h}")
logger.info(f"UA_c: {UA_c}")
logger.info(f"Vi_h: {Vi_h}")
logger.info(f"Vi_c: {Vi_c}")

In [5]:
# From 20230714, to test if model fits current data
UA_h = [0.0069818 , 0.00584034, 0.03041486]
UA_c = [0.01396848, 0.0001    , 0.02286885]
Vi_h = [5.94771006, 4.87661781, 2.19737023]
Vi_c = [5.33410037, 7.56470594, 0.90547187]

# Calibration results obtained for 20230707 - 20230710. Obtained at 20240418 using estimated heat exchanger secondary flow (qhx,s / qts,src)
# UA_h = [0.0001     0.0001     0.01914157]
# UA_c = [0.0001     0.0001     0.07671348]
# Vi_h = [6.7837874  2.55808941 1.74069197]
# Vi_c = [10.         10.          2.13297639]


In [6]:
plt_config = {
      # General plot attributes
      "title": "Thermal storage",
      "subtitle": "model validation",
      "height": 1000,
      "width": 1000,
      "margin": {
        "l":20,
        "r":100,
        "t":100,
        "b":20,
        "pad":5
      },
      "vertical_spacing": 0.03,
      "xdomain": [0, 0.85],
      "arrow_xrel_pos": 60, # seconds
    
        # Individual plot attributes
      "plots": {
          "thermal_storage_flows": {
          "title": "<b>Thermal storage</b>",
          "row_height": 1,
          "bg_color": "bg_gray", # bg gray
          "ylabels_left": ["m<sup>3</sup>/h"],
          "ylims_left": 'manual',
    
          "traces_left": [
            {
              "var_id": "qts_dis",
              "name": "q<sub>ts,dis</sub>",
              "mode": "lines",
              "color": "plotly_blue",
              "width": 3,
            },
            {
              "var_id": "qts_dis_original",
              "name": "q<sub>ts,dis,measured</sub>",
              "color": "plotly_blue", "width": 1.5, "dash": "dash"
            },
            {
              "var_id": "qts_src", #"qts_src",
              "name": "q<sub>ts,src</sub>",
              "mode": "lines",
              "color": "plotly_red",
              "width": 3,
            },
            {
              "var_id": "qts_src_original", #"qts_src",
              "name": "q<sub>ts,src,measured</sub>",
              "color": "plotly_red", "dash": "dash", "width": 1.5,
            },
          ],
        },
    
        "thermal_storage_power_balance": {
          "row_height": 1,
          "title": "Power balance",
          "bg_color": "bg_gray", # bg gray
          "ylabels_left": ["kW<sub>th</sub>"],
          "ylims_left": [0, 260],
    
          "traces_left": [
            # Make this a filled trace between source and discharge, where the color is red when there is more discharge than recharge, and green otherwise
            {
              "var_id": "Pts_src",
              "name": "P<sub>ts,src</sub>",
              "mode": "lines",
              "color": "plotly_yellow",
              "width": 3,
              "fill_between": "Pts_dis", # Traces to plot fill between need to be defined next to each other
            },
            {
              "var_id": "Pts_dis",
              "name": "P<sub>ts,dis</sub>",
              "mode": "lines",
              "color": "wct_purple",
              "width": 3,
              "fill_between": "Pts_src", # Traces to plot fill between need to be defined next to each other
            },
          ]
        },
    
        "temperature_hot_top": {
          "row_height": 1,
          "title": f"Hot - top, UA={UA_h[0]:.2e} W/K, V={Vi_h[0]:.2f} m³",
          "bg_color": "bg_blue", # bg gray
          # "ylabels_left": ["T<sub>wct</sub> (ºC)"],
          "ylabels_left": ["⁰C"],
          "ylims_left": [60, 100],
          "tight_vertical_spacing": True,
    
          "traces_left": [
            # Hot tank
            {"var_id": "Tts_h_t", "mode": "lines", "color": "plotly_red", "width": 3,},
           
            # From solar field
            {"var_id": "Tts_h_in", "mode": "markers", "color": "plotly_yellow", "width": 3, "opacity": 0.5,
              "conditional":{ "var_id": "qts_src", "operator": ">", "threshold_value": 1,}
            },
    
            # To MED
            {"var_id": "Tts_h_out", "mode": "markers", "color": "wct_purple", "width": 3, "opacity": 0.5,
              "conditional":{ "var_id": "qts_dis", "operator": ">", "threshold_value": 1,}
            },        
          ]
        },
          
        "temperatures_hot_med": {
          "row_height": 1,
          "title": f"Hot - medium, UA={UA_h[1]:.2e} W/K, V={Vi_h[1]:.2f} m³",
          "bg_color": "bg_blue", # bg gray
          # "ylabels_left": ["T<sub>wct</sub> (ºC)"],
          "ylabels_left": ["⁰C"],
          "ylims_left": 'manual',
          "tight_vertical_spacing": True,
    
          "traces_left": [
            # Hot tank
            {"var_id": "Tts_h_m", "mode": "lines", "color": "plotly_red", "width": 3,},
          ]
        },
    
        "temperatures_hot_bottom": {
          "row_height": 1,
          "title": f"Hot - bottom, UA={UA_h[2]:.2e} W/K, V={Vi_h[2]:.2f} m³",
          "bg_color": "bg_blue", # bg gray
          # "ylabels_left": ["T<sub>wct</sub> (ºC)"],
          "ylabels_left": ["⁰C"],
          "ylims_left": 'manual',
    
          "traces_left": [
            # Hot tank
            {"var_id": "Tts_h_b", "mode": "lines", "color": "plotly_red", "width": 3,},
          ]
        },
          
    
        "temperatures_cold_top": {
          "row_height": 1,
          "title": f"Cold - top, UA={UA_c[0]:.2e} W/K, V={Vi_c[0]:.2f} m³",
          "bg_color": "bg_blue", # bg gray
          # "ylabels_left": ["T<sub>wct</sub> (ºC)"],
          "ylabels_left": ["⁰C"],
          "ylims_left": 'manual',
          "tight_vertical_spacing": True,
    
          "traces_left": [
            # Hot tank
            {"var_id": "Tts_c_t", "mode": "lines", "color": "plotly_blue", "width": 3,},
          ]
        },
    
        "temperatures_cold_med": {
          "row_height": 1,
          "title": f"Cold - Medium, UA={UA_c[1]:.2e} W/K, V={Vi_c[1]:.2f} m³",
          "bg_color": "bg_blue", # bg gray
          # "ylabels_left": ["T<sub>wct</sub> (ºC)"],
          "ylabels_left": ["⁰C"],
          "ylims_left": 'manual',
          "tight_vertical_spacing": True,
    
          "traces_left": [
            # Hot tank
            {"var_id": "Tts_c_m", "mode": "lines", "color": "plotly_blue", "width": 3,},
          ]
        },
          
        "temperatures_cold_bottom": {
          "row_height": 1,
          "title": f"Cold - bottom, UA={UA_c[2]:.2e} W/K, V={Vi_c[2]:.2f} m³",
          "bg_color": "bg_blue", # bg gray
          # "ylabels_left": ["T<sub>wct</sub> (ºC)"],
          "ylabels_left": ["⁰C"],
          "ylims_left": 'manual',
    
          "traces_left": [
            {"var_id": "Tts_c_b", "mode": "lines", "color": "plotly_blue", "width": 3,},
            # From MED
            {"var_id": "Tts_c_in", "mode": "markers", "color": "wct_purple", "width": 3, "opacity": 0.5,
              "conditional":{ "var_id": "qts_dis", "operator": ">", "threshold_value": 1,}
            },
          ]
        },
    }
}

In [28]:
# Save to json
# import json
# 
# with open( Path("../data") / 'plt_config_thermal_storage.json', 'w') as f:
#     json.dump(plt_config, f, indent=4)

In [6]:
# idx_start = df.index.get_loc(pd.Timestamp('2023-07-09 11:41:00', tz='UTC'))
# idx_end = df.index.get_loc(pd.Timestamp('2023-07-09 13:55:00', tz='UTC'))

### Evaluate model

In [6]:
idx_start = 0
idx_end = len(df)

In [ ]:
# Evaluate model
idx_start = 0
idx_end = len(df)

Ti_ref = np.concatenate((df[Th_labels].values[idx_start+1:], df[Tc_labels].values[idx_start+1:]), axis=1)

N = len(Th_labels)

df_mod = pd.DataFrame()

Tts_h = np.zeros((idx_end-idx_start+1, N), dtype=float)
Tts_c = np.zeros((idx_end-idx_start+1, N), dtype=float)

Tts_h[0] = np.array( [df.iloc[idx_start][T] for T in Th_labels] )
Tts_c[0] = np.array( [df.iloc[idx_start][T] for T in Tc_labels] )

for idx in range(idx_start, idx_end):
    
    j = idx-idx_start+1
    ds = df.iloc[idx]
    
    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
    
    Tts_h[j], Tts_c[j] = thermal_storage_two_tanks_model(
        Ti_ant_h=Tts_h[j-1], Ti_ant_c=Tts_c[j-1],  # [ºC], [ºC]
        Tt_in=ds["Tts_h_in"],  # ºC
        Tb_in=ds["Tts_c_in"],  # ºC
        Tamb=ds["Tamb"],  # ºC

        qsrc=ds["qts_src"],  # m³/h
        qdis=ds["qts_dis"],  # m³/h

        UA_h=UA_h,  # W/K
        UA_c=UA_c,  # W/K
        Vi_h=Vi_h,  # m³
        Vi_c=Vi_c,  # m³
        ts=sample_rate_numeric, Tmin=60  # seg, ºC
    )
    
    out = {label: Tts_h[j][i] for i, label in enumerate(Th_labels)}
    out.update({label: Tts_c[j][i] for i, label in enumerate(Tc_labels)})
    
    result = pd.DataFrame(out, index=[0])
    
    # logger.info(f"Finished iteration {idx} / {idx_end}, elapsed time: {time.time()-start_time:.2f} s, Th_t = {Tts_h[j][0]:.2f} ºC, Tc_t = {Tts_c[j][0]:.2f} ºC")
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)
    

In [ ]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

fig = experimental_results_plot(plt_config, df, df_comp=[df_mod], vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'heat_exhanger_validation_{df.index[0].strftime("%Y%m%d")}')
)

### Evaluate model with different sample rates using util function `evaluate_model`

In [7]:
from solarmed_modeling.thermal_storage import ModelParameters
from solarmed_modeling.thermal_storage.utils import evaluate_model
from solarmed_modeling.visualization.benchmark import visualize_benchmark

model_params = ModelParameters(UA_h=UA_h, V_h =Vi_h, UA_c=UA_c, V_c =Vi_c)

df_mods = []
stats_total = []
# for ts in [5, 300, 600]:
#     df_ = df.copy().resample(f"{ts}s").mean()
#     df_mod, stats = evaluate_model(df_, ts, model_params, alternatives_to_eval=["constant-water-props"])
#     df_mod2, stats2 = evaluate_model(df_, ts, model_params, alternatives_to_eval=["standard"])
    
for ts in [5, 300, 600]:
    df_ = df.copy().resample(f"{ts}s").mean()
    df_mod, stats = evaluate_model(df_, ts, model_params, alternatives_to_eval=["constant-water-props", "standard"])
    # df_mod2, stats2 = evaluate_model(df_, ts, model_params, alternatives_to_eval=["standard"])
    
    df_mods.extend(df_mod) # +df_mod2
    stats_total.extend(stats) # +stats2

# print(f"performance_increase={stats2[0]['elapsed_time']/stats[0]['elapsed_time']:.2f}X")

# fig = experimental_results_plot(
#     plt_config,
#     df_,
#     df_comp=[df_mod, df_mod2],
#     vars_config=vars_config,
#     resample=resample_figures,
# )

# fig.show(
#     config=generate_plotly_config(
#         fig, figure_name=f'thermal_storage_model_validation_{df.index[0].strftime("%Y%m%d")}'
#     )
# )

2024-10-09 08:06:12.625 | INFO     | solarmed_modeling.thermal_storage:evaluate_model:367 - Starting evaluation of alternative constant-water-props. Sample rate = 5 s


KeyboardInterrupt: 

In [ ]:
from solarmed_modeling.thermal_storage import ModelParameters
from solarmed_modeling.thermal_storage.utils import evaluate_model

model_params = ModelParameters(UA_h=UA_h, V_h =Vi_h, UA_c=UA_c, V_c =Vi_c)

sample_rates: list[int] = [5, 30, 60, 300, 600, 1000]
dfs = [df.copy().resample(f"{ts}s").mean() for ts in sample_rates] 

dfs_mod: list[pd.DataFrame] = []
stats = []
for df_, ts in zip(dfs, sample_rates):
    out = evaluate_model(df_, ts, model_params, alternatives_to_eval=["standard", "constant-water-props"])
    dfs_mod.extend(out[0])
    stats.extend( out[1] )
    
# Match sample rates so they can be plot together
dfs_mod = [df_.reindex(df.index, method='ffill') for df_ in dfs_mod]

plt_config["title"] = (
    f"<b>Thermal storage</b> model validation {df.index[0].strftime('%d/%m/%Y')}"
)
# plt_config["subtitle"] = (
#     f"β: {beta:.4e}, H: {H:.4f}, ɣ: {gamma:.4f}, Ts={sample_rates}s"
# )

fig = experimental_results_plot(
    plt_config,
    df,
    df_comp=dfs_mod,
    vars_config=vars_config,
    resample=resample_figures,
)

fig.show(
    config=generate_plotly_config(
        fig, figure_name=f'thermal_storage_model_validation_{df.index[0].strftime("%Y%m%d")}'
    )
)

In [ ]:
# Save figure
from datetime import datetime

save_figure(
    figure_name=f"thermal_storage_validation_{df.index[0].strftime('%Y%m%d')}_generated_{datetime.now().strftime('%Y%m%dT%H%M')}", 
    figure_path=output_path,
    fig=fig, formats=('svg', 'html', 'png'), 
    width=fig.layout.width, 
    height=fig.layout.height, 
    scale=2
)

In [ ]:
figs = visualize_benchmark(stats, title = "<b>Benchmark results</b><br>For the thermal storage model</br>", output_unit="ºC", width=500)
[fig.show() for fig in figs]

# Old

In [ ]:
# Matplotlib visualizations settings
import seaborn as sns
import matplotlib.dates as mdates

sns.set_theme()
myFmt = mdates.DateFormatter('%H:%M')
plot_colors = sns.color_palette()

locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
formatter.formats = ['%y',  # ticks are mostly years
                     '%b',       # ticks are mostly months
                     '%d',       # ticks are mostly days
                     '%H:%M',    # hrs
                     '%H:%M',    # min
                     '%S.%f', ]  # secs
# these are mostly just the level above...
formatter.zero_formats = [''] + formatter.formats[:-1]
# ...except for ticks that are mostly hours, then it is nice to have
# month-day:
formatter.zero_formats[3] = '%d-%b'

formatter.offset_formats = ['',
                            '%Y',
                            '%b %Y',
                            '%d %b %Y',
                            '%d %b %Y',
                            '%d %b %Y %H:%M', ]

In [41]:
# Load data

# Load variables information
with open( base_path / 'data/variables_config.hjson') as f:
    vars_config = hjson.load(f)

# Read data from csv, the index column is the one named "time", which is not the first one
df = pd.read_csv(data_path, parse_dates=True, index_col='TimeStamp')

# Rename index column to "time"
df.index.names = ['time']

# Set UTC timezone
df = df.tz_localize('UTC')

display(df.head())

In [42]:
# Preprocessing
from phd_visualizations.utils import rename_signal_ids_to_var_ids
from phd_visualizations.utils.units import unit_conversion

%autoreload 2

# Sample every `sample_rate` seconds to reduce the size of the dataframe
df = df.resample(sample_rate).mean()

# Rename columns from signal_id to var_id
df = rename_signal_ids_to_var_ids(df, vars_config)

# Flow meter of Qdis is broken, using the output of the three-way valve 
# model as an alternative
# data_Qdis = get_Q_from_3wv_model(datos_date_str, sample_rate_str=sample_rate_str)

# df["Tts_b_in"] = df["Tts_b_in"].values() if 'Tts_b_in' in df.columns else np.zeros(len(data))

# Merge both dataframes so they are synced in time
# data.rename(columns={'m_ts_dis': 'm_ts_dis_sensor'}, inplace=True) # Rename the invalid signal
# data = pd.merge(data, data_Qdis, left_index=True, right_index=True, how='outer')

# Convert units
df = unit_conversion(df, vars_config, input_unit_key='units_scada', output_unit_key='units_model')

display(df.head())

In [43]:
logger.debug(list(df.columns))

In [50]:
# Visualize data
# TODO: Temporal, once protoyped move to phd_visualizations

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from phd_visualizations.constants import color_palette

idxs_src = (df["qts_src"] > 5).values
idxs_dis = (df["qts_dis"] > 0.1).values if "qts_dis" in df.columns else None

color_src = "#c01c28"
color_dis = "#1c71d8"

colors = [color_palette['cool_red'], color_palette['cool_green']]

# Create a figure and axis
# fig, ax = plt.subplots()
fig = plt.figure(figsize=(8,10),) #constrained_layout=True)
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])

ax = fig.add_subplot(gs[0])

# Plot each temperature signal
for var_name in ['Tts_h_t', 'Tts_h_m', 'Tts_h_b', 'Tts_c_t', 'Tts_c_m', 'Tts_c_b',]:
    if var_name not in ["time", "Tts_h_t_in", "Tts_c_b_in"] and var_name.startswith('Tts'):
        if var_name.startswith('Tts_h'):
            color = colors[0]
        else:
            color = colors[1]
            
        if var_name.endswith('t'):
            line_type = 'solid'
        elif var_name.endswith('m'):
            line_type = 'dashed'
        else:
            line_type = 'dashdot'
            
        var_label = vars_config[var_name]['label_latex']
        
        ax.plot(df.index, df[var_name], linestyle=line_type, label=var_label, color=color)
    
# Source temperature
ax.plot(df.iloc[idxs_src].index, df.iloc[idxs_src]["Tts_h_t_in"], '.',color=color_src, alpha=0.3, label=vars_config['Tts_h_t_in']['label_latex'], zorder=1)
# Discharge temperature
ax.plot(df.index[idxs_dis], df["Tts_c_b_in"][idxs_dis], '.',color=color_dis, alpha=0.3, label=vars_config['Tts_c_b_in']['label_latex'], zorder=1)

ax.set_xticks([]); ax.set_xticklabels([])

# Ambient temperature
ax_r = ax.twinx()
ax_r.set_axisbelow(True)
ax_r.plot(df.index, df['Tamb'], label='$T_{amb}$ (right)', alpha=0.3)
ax_r.set_ylim([0, 100])
ax_r.set_yticks([df['Tamb'].min(), df['Tamb'].max()])
# ax_r.tick_params(axis='both', which='both', zorder=-10)

# Remove x-axis labels
ax.set_xticklabels([])
ax_r.set_xticklabels([])

# Set labels and title
ax.set_ylabel('Temperature ($^{\circ}C$)')
ax.set_title('Thermal storage temperature profile evolution over time', fontweight='bold')

# Add legend
ax.legend(ncols=4)
# ax.tick_params(axis='x', which='both',
#                 bottom=False) # turn off major & minor ticks on the bottom
# x0 = round(193_000/ts)
# duration = round(20*3600/ts)
# xrange=[x0, round(x0+duration)]
# ax.axvspan(df.index[xrange[0]], df.index[xrange[1]], alpha=0.3)
# Remove x-axis label


# Flows plot
ax = fig.add_subplot(gs[1], sharex=ax)

ax.plot(df.index, df["qts_src"], color=color_src, label=vars_config["qts_src"]["label_latex"])
# ax.legend()
# ax.set_ylabel(f'{var_labels["qts_src"]} (L/min)')

# ax = ax.twinx()
# ax.set_axisbelow(True)
ax.plot(df.index, df["qts_dis"], color=color_dis, label=vars_config["qts_dis"]["label_latex"])
ax.legend()
ax.set_ylabel('Flow (m³/h)')


ax.set_xlabel('Time')
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

# fig.set_constrained_layout_pads(hspace=0.0, h_pad=0.0)  
# Adjust the spacing between subplots
plt.subplots_adjust(top=0.940, bottom=0.075)

# Display the plot
plt.show()

## Test model prior to parameter fit

In [ ]:
from models_psa import thermal_storage_model_two_tanks, thermal_storage_model_single_tank
from parameters_fit import calculate_iae, calculate_ise, calculate_itae
from visualization.calibrations import plot_model_result_thermal_storage


# Parameters
Tmin = 60 # Minimum useful temperature, °C
N = 3 # Number of volumes
V = 15 # Volume of an individual tank, m³
Vt = 15*2 # Total volume of the storage system (V tank·N tanks)
Tmin = 60 # Minimum useful temperature, °C
V_i = np.ones(N)*V/N  # Volume of each control volume

# Model parameters
UA_h  = np.array([0.0068, 0.004, 0.0287])
# Vi_h = np.array([2.9722, 1.7128, 9.4346])
Vi_h  = V_i.copy()

UA_c  = np.array([0.0068, 0.004, 0.0287])
# Vi_c = np.array([2.9722, 1.7128, 9.4346])
Vi_c = V_i.copy()

# Inputs 

# Since we are using the first output as starting point, start from the second value
Ti_ant_h = np.array( [data[T][0] for T in Tin_labels_h] )
Ti_ant_c = np.array( [data[T][0] for T in Tin_labels_c] )
Tt_in = data.Tts_t_in.values[1:]
Tb_in = data.Tts_b_in.values[1:] if 'Tts_b_in' in data.columns else np.zeros(len(data)-1)

Tamb = data.Tamb.values[1:]
Qsrc = data.mts_src.values[1:] if 'mts_src' in data.columns else np.zeros(len(data)-1)
Qdis = data.mts_dis.values[1:] if 'mts_dis' in data.columns else np.zeros(len(data)-1)


msrc = np.zeros(len(data)-1, dtype=float); mdis = np.zeros(len(data)-1, dtype=float)
for idx in range(1, len(data)-1):
    msrc[idx] = Qsrc[idx]/60*w_props(P=0.1, T=Tt_in[idx]+273.15).rho*1e-3 # rho [kg/m³] # Convertir L/min a kg/s
    mdis[idx] = Qdis[idx]*w_props(P=0.1, T=data.Tts_h_t[idx]+273.15).rho*1e-3 # rho [kg/m³] # Convertir L/s a kg/s

# Experimental outputs
Ti_ref = np.concatenate((data[Tin_labels_h].values[1:], data[Tin_labels_c].values[1:]), axis=1)

# Initialize result vectors
Ti_h_mod   = np.zeros((len(data)-1, N), dtype=float)
Ti_c_mod   = np.zeros((len(data)-1, N), dtype=float)


# Evaluate model
for idx in range(len(data)-1):
    # Ti_c_mod[idx] = thermal_storage_model_single_tank(
    #                              Ti_ant_c, Tt_in=0, Tb_in=Tb_in[idx], Tamb=Tamb[idx], 
    #                              mt_in=0, mb_in=mdis[idx], mt_out=mdis[idx]-msrc[idx], mb_out=msrc[idx],
    #                              UA=UA_c, V_i=Vi_c, 
    #                              N=3, ts=ts, calculate_energy=False)
    Ti_h_mod[idx], Ti_c_mod[idx] = thermal_storage_model_two_tanks(
                                        Ti_ant_h=Ti_ant_h, Ti_ant_c=Ti_ant_c, 
                                        Tt_in=Tt_in[idx], 
                                        Tb_in= Tb_in[idx], 
                                        Tamb= Tamb[idx], 
                                        msrc= msrc[idx], 
                                        mdis= mdis[idx], 
                                        UA_h=UA_h, UA_c=UA_c,
                                        Vi_h=Vi_h, Vi_c=Vi_c,
                                        ts=ts, Tmin=Tmin, V=Vt, calculate_energy=False)
    Ti_ant_h = Ti_h_mod[idx]
    Ti_ant_c = Ti_c_mod[idx]
    
# Calculate performance metrics
Ti_mod = np.concatenate((Ti_h_mod, Ti_c_mod), axis=1)
iae  = calculate_iae(Ti_mod, Ti_ref)
ise  = calculate_ise(Ti_mod, Ti_ref)
itae = calculate_itae(Ti_mod, Ti_ref)

Ti_mod = Ti_c_mod
Ti_ref = data[Tin_labels_c].values[1:]
iae  = calculate_iae(Ti_mod, Ti_ref)
ise  = calculate_ise(Ti_mod, Ti_ref)
itae = calculate_itae(Ti_mod, Ti_ref)

# Visualize result
plot_model_result_thermal_storage(N*2, Tin_labels, data, np.concatenate((Ti_h_mod,Ti_c_mod), axis=1), 
                                  np.concatenate((UA_h, UA_c)), np.concatenate((Vi_h, Vi_c)), 
                                  itae, iae, ise)

In [ ]:

from parameters_fit import objective_function
from models_psa import thermal_storage_model_two_tanks
from visualization.calibrations import plot_model_result_thermal_storage
from optimparallel import minimize_parallel
from parameters_fit import calculate_iae, calculate_ise, calculate_itae

save_figure = True
figure_path = '/home/jmserrano/Nextcloud/Juanmi_MED_PSA/EURECAT/Modelos/attachments'
figure_name = 'result_model_ts_calibration'


# Parameters
Tmin = 60 # Minimum useful temperature, °C
N = 3 # Number of volumes
V = 15 # Volume of an individual tank, m³
Vt = 15*2 # Total volume of the storage system (V tank·N tanks)
V_i = np.ones(N)*V/N  # Volume of each control volume

# Inputs 

# Since we are using the first output as starting point, start from the second value
Ti_ant_h = np.array( [data[T][0] for T in Tin_labels_h] )
Ti_ant_c = np.array( [data[T][0] for T in Tin_labels_c] )
Tt_in = data.Tts_t_in.values[1:]
Tb_in = data.Tts_b_in.values[1:] if 'Tts_b_in' in data.columns else np.zeros(len(data)-1)

Tamb = data.Tamb.values[1:]
Qsrc = data.mts_src.values[1:] if 'mts_src' in data.columns else np.zeros(len(data)-1)
Qdis = data.mts_dis.values[1:] if 'mts_dis' in data.columns else np.zeros(len(data)-1)


msrc = np.zeros(len(data)-1, dtype=float); mdis = np.zeros(len(data)-1, dtype=float)
for idx in range(1, len(data)-1):
    msrc[idx] = Qsrc[idx]/60*w_props(P=0.1, T=Tt_in[idx]+273.15).rho*1e-3 # rho [kg/m³] # Convertir L/min a kg/s
    mdis[idx] = Qdis[idx]*w_props(P=0.1, T=data.Tts_h_t[idx]+273.15).rho*1e-3 # rho [kg/m³] # Convertir L/s a kg/s

# Experimental outputs
Ti_ref = np.concatenate((data[Tin_labels_h].values[1:], data[Tin_labels_c].values[1:]), axis=1)

# Define optimizer inputs
inputs = [Ti_ant_h, Ti_ant_c, Tt_in, Tb_in, Tamb, msrc, mdis]  # Input values
outputs = Ti_ref  # Actual output values
params = (ts, Tmin, Vt)    # Constant model parameters
params_objective_function = {'metric': 'IAE', 'recursive':True, 'n_outputs':2, 
                             'n_parameters': 4} # 'len_outputs':[N, N]

# Set initial parameter values
# initial_parameters = [0.01 for _ in range(N)]
initial_parameters = np.concatenate((np.array([0.02136564, 0.01593324, 0.01918577]), # UA_h
                                     np.array([0.02136564, 0.01593324, 0.01918577]), # UA_c
                                     np.ones(N)*V/N, # Vi_h,
                                     np.ones(N)*V/N, # Vi_c
                                    ))
#         UAmin, UAmax    UAmin, UAmax    Vi_min     Vi_max       Vi_min     Vi_max
bounds = ((1e-4, 1),)*N + ((1e-4, 1),)*N + ((0.1*V/N, 2*V/N),)*N + ((0.1*V/N, 2*V/N),)*N

# Perform parameter calibration
optimized_parameters = minimize_parallel(
    objective_function,
    initial_parameters,
    args=(thermal_storage_model_two_tanks, inputs, outputs, params, params_objective_function),
    bounds = bounds,
    # method='L-BFGS-B'
).x

op = optimized_parameters

L = int(len(op)/4)
UA_h  = op[:L]
UA_c  = op[L:2*L]
Vi_h  = op[2*L:3*L]
Vi_c  = op[3*L:]

# optimized_parameters = array([6.77724155e-03, 3.96580419e-03, 2.87258611e-02, 6.88542188e-03, 2.97217468e+00, 1.71277001e+00, 9.43455760e+00, 3.78073750e+00])
# Run model with optimized parameters

# Reset initial input
Ti_ant_h = np.array( [data[T][0] for T in Tin_labels_h] )
Ti_ant_c = np.array( [data[T][0] for T in Tin_labels_c] )

# Initialize result vectors
Ti_h_mod   = np.zeros((len(data)-1, N), dtype=float)
Ti_c_mod   = np.zeros((len(data)-1, N), dtype=float)


# Evaluate model
for idx in range(len(data)-1):
    # Ti_c_mod[idx] = thermal_storage_model_single_tank(
    #                              Ti_ant_c, Tt_in=0, Tb_in=Tb_in[idx], Tamb=Tamb[idx], 
    #                              mt_in=0, mb_in=mdis[idx], mt_out=mdis[idx]-msrc[idx], mb_out=msrc[idx],
    #                              UA=UA_c, V_i=Vi_c, 
    #                              N=3, ts=ts, calculate_energy=False)
    Ti_h_mod[idx], Ti_c_mod[idx] = thermal_storage_model_two_tanks(
                                        Ti_ant_h=Ti_ant_h, Ti_ant_c=Ti_ant_c, 
                                        Tt_in=Tt_in[idx], 
                                        Tb_in= Tb_in[idx], 
                                        Tamb= Tamb[idx], 
                                        msrc= msrc[idx], 
                                        mdis= mdis[idx], 
                                        UA_h=UA_h, UA_c=UA_c,
                                        Vi_h=Vi_h, Vi_c=Vi_c,
                                        ts=ts, Tmin=Tmin, V=Vt, calculate_energy=False)
    Ti_ant_h = Ti_h_mod[idx]
    Ti_ant_c = Ti_c_mod[idx]
    
# Calculate performance metrics
Ti_mod = np.concatenate((Ti_h_mod, Ti_c_mod), axis=1)
iae  = calculate_iae(Ti_mod, Ti_ref)
ise  = calculate_ise(Ti_mod, Ti_ref)
itae = calculate_itae(Ti_mod, Ti_ref)

# Ti_mod = Ti_c_mod
# Ti_ref = data[Tin_labels_c].values[1:]
# iae  = calculate_iae(Ti_mod, Ti_ref)
# ise  = calculate_ise(Ti_mod, Ti_ref)
# itae = calculate_itae(Ti_mod, Ti_ref)

# Visualize result

# Since we don't have fill it with zeros
if "mts_dis" not in data.columns:
    data["mts_dis"] = np.zeros(len(data))

plot_model_result_thermal_storage(N*2, Tin_labels, data, np.concatenate((Ti_h_mod,Ti_c_mod), axis=1), 
                                  np.concatenate((UA_h, UA_c)), np.concatenate((Vi_h, Vi_c)), 
                                  itae, iae, ise, 
                                  save_figure=save_figure, figure_path=figure_path, 
                                  figure_name=f'{figure_name}_{data.index[0].to_pydatetime().date().isoformat()}')


"""
Calibrated parameters:
    
    - UA_h: [0.00561055, 0.00225925, 0.04767485]
    - UA_c: [0.01019435, 0.00299455, 0.11281388]
    - Vi_h: [2.44754599, 4.86137431, 2.4105236 ]
    - Vi_c: [4.50502171,  1.33711331, 10.      ]
    
    
    V2. 20230714
    
    - UA_h: [0.0069818 , 0.00584034, 0.03041486]
    - UA_c: [0.01396848, 0.0001    , 0.02286885]
    - Vi_h: [5.94771006, 4.87661781, 2.19737023]
    - Vi_c: [5.33410037, 7.56470594, 0.90547187]
"""